In [0]:
# -*- coding: utf-8 -*-
#-*- coding: cp950 -*-
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from typing import *

import torch
import torch.optim as optim

%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.vision import *
from fastai.text import *
from fastai.callbacks import *
from fastai.core import *

import tensorflow as tf
import random as rn
np.random.seed(0)
rn.seed(0)

import keras
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
# Importing sklearn libraries
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
import pickle

from keras.preprocessing import image
from sklearn.model_selection import train_test_split

# Importing Keras libraries
from keras.utils import np_utils
from keras.models import Sequential
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import Dense, Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D
 
import warnings
warnings.filterwarnings('ignore')
import cv2

In [0]:
os.listdir()

['.ipynb_checkpoints',
 '00Food-11K  Recognition VGG16 OK .ipynb',
 'AI 李宏毅 - HackMD UV.pdf',
 'AI 李宏毅 - HackMD UV2.pdf',
 'cosmos',
 'sakura',
 'simple CNN.ipynb',
 'Transfer Learning  CNN計算 J.pdf',
 'Transfer Learning  feature extraction logistic J.pdf',
 'Transfer Learning  GBT J.pdf',
 'Transfer Learning  logistic J UV.pdf',
 'Transfer Learning  logistic J.pdf',
 'Transfer Learning  傅立葉計算1 J.pdf',
 'Transfer Learning  傅立葉計算2 J.pdf',
 'Transfer Learning  傅立葉計算3 J.pdf',
 'Transfer Learning  櫻花與波斯菊 J.pdf',
 'Transfer Learning  櫻花與波斯菊數據 J.pdf',
 'Transfer Learning  權重可視化 J.pdf',
 'Transfer Learning  權重可視化 J2.pdf']

In [11]:
#在colab下接入google drive
from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)
base_dir = '/content/gdrive/My Drive'
#path = Path(base_dir +'/imgs4')  #imgs
path = Path(base_dir +'/DLtry')
path.mkdir(parents=True,exist_ok=True)
os.chdir(path)

Mounted at /content/gdrive


In [0]:
#分割數據與設定參數
input_shape = (224, 224, 3)
batch_size = 128
epochs =12 # 100
num_classes = 2  #######

x = []
y = []

for f in os.listdir("sakura"):
    x.append(image.img_to_array(image.load_img("sakura/"+f, target_size=input_shape[:2])))
    y.append(0)
for f in os.listdir("cosmos"):
    x.append(image.img_to_array(image.load_img("cosmos/"+f, target_size=input_shape[:2])))
    y.append(1)
x = np.asarray(x)
x /= 255
y = np.asarray(y)
y = keras.utils.to_categorical(y, num_classes)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state= 3)

In [0]:
#VGG16のモデル・パラメータを読み込みます。畳み込み層、プーリング層の出力を特徴量として使用するために、
#末尾の全結合層は除き、最後に平均値でプーリングします。
from keras.models import Sequential  #必須適時呼叫***
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

from keras.applications.vgg16 import VGG16
from keras.models import Model

#base_model = VGG16(weights='imagenet', include_top=False)
base_model = VGG16(weights='imagenet', include_top=False, pooling="avg")

#訓練データをVGG16に処理させたものを、新しい特徴量とします。
x_train_vgg16 = base_model.predict(x_train)
x_test_vgg16 = base_model.predict(x_test)

In [14]:
#線形SVMに訓練データを学習させます。
from sklearn.svm import SVC
clf = SVC(kernel="linear").fit(x_train_vgg16, y_train)

#精度
from sklearn.metrics import accuracy_score
print("acc = {}".format(accuracy_score(clf.predict(x_train_vgg16), y_train)))
print("val_acc = {}".format(accuracy_score(clf.predict(x_test_vgg16), y_test)))

ValueError: ignored

In [0]:
訓練データ、バリデーションデータともに、100%の精度を達成できています

In [21]:
#######################
#RF
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=45,n_estimators=220,random_state=230).fit(x_train_vgg16, y_train)

#精度
from sklearn.metrics import accuracy_score
print("acc = {}".format(accuracy_score(clf.predict(x_train_vgg16), y_train)))
print("val_acc = {}".format(accuracy_score(clf.predict(x_test_vgg16), y_test)))

acc = 1.0
val_acc = 0.9


In [24]:
print(clf.predict([[0, 0]]))

ValueError: ignored

In [25]:
clf.feature_importances_

array([0.000842, 0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.030343, 0.003535])

In [27]:
list(zip(x_train, clf.feature_importances_))

[(array([[[0.317647, 0.482353, 0.152941],
          [0.329412, 0.494118, 0.156863],
          [0.333333, 0.501961, 0.14902 ],
          [0.333333, 0.509804, 0.145098],
          ...,
          [0.207843, 0.388235, 0.101961],
          [0.207843, 0.376471, 0.098039],
          [0.227451, 0.396078, 0.117647],
          [0.262745, 0.431373, 0.160784]],
  
         [[0.313726, 0.478431, 0.14902 ],
          [0.32549 , 0.490196, 0.152941],
          [0.333333, 0.501961, 0.152941],
          [0.333333, 0.509804, 0.152941],
          ...,
          [0.211765, 0.392157, 0.105882],
          [0.207843, 0.376471, 0.098039],
          [0.235294, 0.403922, 0.12549 ],
          [0.266667, 0.435294, 0.164706]],
  
         [[0.313726, 0.478431, 0.14902 ],
          [0.317647, 0.482353, 0.145098],
          [0.329412, 0.498039, 0.14902 ],
          [0.329412, 0.505882, 0.14902 ],
          ...,
          [0.223529, 0.392157, 0.109804],
          [0.223529, 0.384314, 0.109804],
          [0.235294, 0.

In [28]:
pd.DataFrame(list(zip(x_train, clf.feature_importances_)))   #PCA,NMF

,0,1
0,"[[[0.31764707, 0.48235294, 0.15294118], [0.329...",0.000842
1,"[[[0.35686275, 0.54901963, 0.70980394], [0.364...",0.000000
2,"[[[0.9607843, 0.9254902, 0.8980392], [0.972549...",0.000000
3,"[[[0.9019608, 0.9411765, 0.98039216], [0.90196...",0.000000
4,"[[[0.08235294, 0.03137255, 0.007843138], [0.07...",0.000000
5,"[[[0.6627451, 0.5411765, 0.36078432], [0.66666...",0.000000
6,"[[[0.007843138, 0.29411766, 0.6039216], [0.011...",0.000000
7,"[[[0.29803923, 0.44705883, 0.6901961], [0.3058...",0.000874
8,"[[[0.2, 0.11764706, 0.28627452], [0.3254902, 0...",0.000000
9,"[[[0.8352941, 0.5294118, 0.62352943], [0.78039...",0.000000


In [16]:
##################################
#logistic
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(class_weight='balanced', multi_class="auto", max_iter=200, random_state=1).fit(x_train_vgg16, y_train)

#精度
from sklearn.metrics import accuracy_score
print("acc = {}".format(accuracy_score(clf.predict(x_train_vgg16), y_train)))
print("val_acc = {}".format(accuracy_score(clf.predict(x_test_vgg16), y_test)))

ValueError: ignored

In [18]:
#################
#GB
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(random_state=230).fit(x_train_vgg16, y_train)

#精度
from sklearn.metrics import accuracy_score
print("acc = {}".format(accuracy_score(clf.predict(x_train_vgg16), y_train)))
print("val_acc = {}".format(accuracy_score(clf.predict(x_test_vgg16), y_test)))

ValueError: ignored

In [0]:
from sklearn.tree import DecisionTreeClassifier  ## decision tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
import xgboost

In [20]:
clf = XGBClassifier(random_state=230).fit(x_train_vgg16, y_train)

#精度
from sklearn.metrics import accuracy_score
print("acc = {}".format(accuracy_score(clf.predict(x_train_vgg16), y_train)))
print("val_acc = {}".format(accuracy_score(clf.predict(x_test_vgg16), y_test)))

ValueError: ignored